In [32]:
from __future__ import print_function

import json
import time

import re
import requests

In [23]:
headers = 'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; Touch; rv:11.0) like Gecko'
# youtube comment scrape
results =[]
comment_list = []
youtube_id ='2HyspUw5L0U'
url = f'https://www.youtube.com/watch?v={youtube_id}'

response = requests.get(f'https://www.youtube.com/watch?v={youtube_id}')
print(url)
print()
print(response)
SORT_BY_POPULAR = 0
SORT_BY_RECENT = 1

YT_CFG_RE = r'ytcfg\.set\s*\(\s*({.+?})\s*\)\s*;'
YT_INITIAL_DATA_RE = r'(?:window\s*\[\s*["\']ytInitialData["\']\s*\]|ytInitialData)\s*=\s*({.+?})\s*;\s*(?:var\s+meta|</script|\n)'

https://www.youtube.com/watch?v=2HyspUw5L0U

<Response [200]>


In [24]:
def regex_search(text,pattern,group=1,default=None):
    match = re.search(pattern,text)
    if match:
        match = match.group(group)
    else:
        default
    return match

In [25]:
def ajax_request(session, endpoint, ytcfg, retries=5, sleep=20):
    url = 'https://www.youtube.com'+endpoint['commandMetadata']['webCommandMetadata']['apiUrl']

    data = {
        'context': ytcfg['INNERTUBE_CONTEXT']
        ,'continuation': endpoint['continuationCommand']['token']
    }
    print(data)
    for _ in range(retries):
        response = session.post(
            url
            ,params = {'key':ytcfg['INNERTUBE_API_KEY']}
            ,json = data
        )
        if response.status_code == 200:
            return response.json()
        if response.status_code in [403,413]:
            return {}
        else:
            time.sleep(time)


In [26]:
def download_comments(sort_by=SORT_BY_RECENT, sleep=.1):
    session = requests.Session()
    session.headers['User-Agent'] = headers

    response = session.get(url)

    if 'uxe=' in response.request.url:
        session.cookies.set('CONSENT','YES+cb',domain='.youtube.com')
        response = session.get(url)

    html = response.text
    ytcfg = json.loads(regex_search(html, YT_CFG_RE, default=''))
    if not ytcfg:
        return # unable to extract configuration

    data = json.loads(regex_search(html,YT_INITIAL_DATA_RE,default=''))

    section = next(search_dict(data,'itemSectionRenderer'),None)
    renderer = next(search_dict(section,'continuationItemRenderer'),None) if section else None
    if not renderer:
        return

    needs_sorting = sort_by != SORT_BY_RECENT
    continuations = [renderer['continuationEndpoint']]
    while continuations:
        continuation = continuations.pop()
        response = ajax_request(session,continuation,ytcfg)

        if not response:
            break
        if list(search_dict(response,'externalErrorMessage')):
            raise RuntimeError('Error returned from server: ' + 
            next(search_dict(response, 'externalErrorMessage')))
        if needs_sorting:
            sort_menu = next(search_dict(response,'sortFilterSubMenuRenderer'), {}).get('subMenuItems',[])
            if sort_by < len(sort_menu):
                continuations = [sort_menu[sort_by]['serviceEndpoint']]
                needs_sorting = False
                continue
            raise RuntimeError('Failed to set sorting')

        actions = list(search_dict(response,'reloadContinuationItemsCommand'))+list(search_dict(response,'appendContinuationItemsAction'))

        for action in actions:
            for item in action.get('continuationItems',[]):
                if action['targetId'] == 'comments-section':
                    continuations[:0] = [ep for ep in search_dict(item,'continuationEndpoint')]
                if action['targetId'].startswith('comment-replies-item') and 'continuationItemRenderer' in item:
                    # 'show more replies' 버튼
                    pass
                    # continuation.append(next(search_dict(item,'buttonRenderer'))['command'])

        for comment in reversed(list(search_dict(response,'commentRenderer'))):
            yield {
                '댓글': ''.join([c['text'] for c in comment['contentText'].get('runs',[])])
                ,'좋아요': comment.get('voteCount',{}).get('simpleText','0')
            }
            
            # return ''.join([c['댓글'] for c in comment['contentText'].get('runs',[])])

        time.sleep(sleep)

In [27]:
def search_dict(partial, search_key):
    stack = [partial]
    while stack:
        current_item = stack.pop()
        # dictionary
        if isinstance(current_item, dict): 
            for k,v in current_item.items():
                if k == search_key:
                    yield v
                else:
                    stack.append(v)
        # list
        elif isinstance(current_item, list):
            for v in current_item:
                stack.append(v)


In [51]:
for item in download_comments():
    print(item)

# \n 제거하려면 .strip()

{'context': {'client': {'hl': 'ko', 'gl': 'KR', 'remoteHost': '119.196.53.160', 'deviceMake': '', 'deviceModel': '', 'visitorData': 'Cgt5bWJxUU5NVWlPVSinpa6JBg%3D%3D', 'userAgent': 'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; Touch; rv:11.0) like Gecko,gzip(gfe)', 'clientName': 'WEB', 'clientVersion': '2.20210827.01.00', 'osName': 'Windows', 'osVersion': '6.3', 'originalUrl': 'https://www.youtube.com/watch?v=2HyspUw5L0U', 'platform': 'DESKTOP', 'clientFormFactor': 'UNKNOWN_FORM_FACTOR', 'browserName': 'IE', 'browserVersion': '11.0'}, 'user': {'lockedSafetyMode': False}, 'request': {'useSsl': True}, 'clickTracking': {'clickTrackingParams': 'IhMIi/Kd9LHW8gIV4pDCCh3ePAoi'}}, 'continuation': 'Eg0SCzJIeXNwVXc1TDBVGAYyJSIRIgsySHlzcFV3NUwwVTAAeAJCEGNvbW1lbnRzLXNlY3Rpb24%3D'}
{'댓글': "the lights during Karina's dance were so cool! Also their live vocals <3", '좋아요': '142'}
{'댓글': 'Giselle looks so gorgeous with black hair💗✨\n\n\n\n\n\n\nI choose the right bias.', '좋아요': '245'}
{'댓글': 'Winte

In [56]:
# with open('8.29_aespa_youtube_comments.csv', "w") as file1:
#     writes = csv.writer(file1, delimiter=' ', quoting=csv.QUOTE_ALL)
#     for item in download_comments():
#         writes.writerow(item.values())


    # writes.writerows(download_comments())